<a href="https://colab.research.google.com/github/prisar/ai/blob/main/project_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###tinygrad

In [5]:
!git clone https://github.com/geohot/tinygrad.git

fatal: destination path 'tinygrad' already exists and is not an empty directory.


In [ ]:
cd tinygrad

In [ ]:
!pip install -e .

In [ ]:
from tinygrad.tensor import Tensor

x = Tensor.eye(3, requires_grad=True)
y = Tensor([[2.0,0,-2.0]], requires_grad=True)
z = y.matmul(x).sum()
z.backward()

print(x.grad.numpy())  # dz/dx
print(y.grad.numpy())  # dz/dy

[[ 2.  2.  2.]
 [ 0.  0.  0.]
 [-2. -2. -2.]]
[[1. 1. 1.]]


In [ ]:
from tinygrad.tensor import Tensor;
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N);
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2);
print((c.numpy() - (a.numpy() @ b.numpy())).mean())

1.7190548e-08


In [ ]:
%%writefile fast.sh

DEBUG=3 OPTLOCAL=1 python3 -c "from tinygrad.tensor import Tensor;
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N);
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2);
print((c.numpy() - (a.numpy() @ b.numpy())).mean())"

Writing fast.sh


In [ ]:
!sh fast.sh

*** exec  0.01 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1024, 1, 1024)                in(1): [(1024, 1024)] (1024, 1, 1024)
*** exec  4.30 GB    0.17 ms op: MovementOps.EXPAND   out(float): (1024, 1024, 1024)             in(1): [(1024, 1, 1024)] (1024, 1024, 1024)
*** exec  4.31 GB    0.10 ms op: MovementOps.PERMUTE  out(float): (1024, 1024)                   in(1): [(1024, 1024)] (1, 0)
*** exec  4.31 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1, 1024, 1024)                in(1): [(1024, 1024)] (1, 1024, 1024)
*** exec  8.60 GB    0.13 ms op: MovementOps.EXPAND   out(float): (1024, 1024, 1024)             in(1): [(1, 1024, 1024)] (1024, 1024, 1024)
*** exec  8.60 GB  420.41 ms op: FusedOps.MULACC      out(float): (1024, 1024, 1)                in(2): [(1024, 1024, 1024)] (1024, 1024, 1)
*** exec  0.02 GB    0.11 ms op: MovementOps.RESHAPE  out(float): (1024, 1024)                   in(1): [(1024, 1024, 1)] (1024, 1024)
3.792106e-09


In [ ]:
!python3 -m pip install -e '.[testing]'

In [ ]:
!python -m pytest

In [4]:
!DEBUG=3 OPTLOCAL=1 python3 -c "from tinygrad.tensor import Tensor; \
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N); \
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2); \
print((c.numpy() - (a.numpy() @ b.numpy())).mean())"

*** exec  0.00 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1, 1)                         in(1): [()] (1, 1)
*** exec  0.00 GB    0.18 ms op: MovementOps.EXPAND   out(float): (1024, 1024)                   in(1): [(1, 1)] (1024, 1024)
*** exec  0.02 GB    0.12 ms op: MovementOps.RESHAPE  out(float): (2, 1024, 1024)                in(1): [(2097152,)] (2, 1024, 1024)
*** exec  0.02 GB    0.10 ms op: MovementOps.SHRINK   out(float): (1, 1024, 1024)                in(1): [(2, 1024, 1024)] ((0, 1), (0, 1024), (0, 1024))
*** exec  0.02 GB    0.09 ms op: MovementOps.RESHAPE  out(float): (1024, 1024)                   in(1): [(1, 1024, 1024)] (1024, 1024)
*** exec  0.02 GB    0.07 ms op: MovementOps.RESHAPE  out(float): (1, 1)                         in(1): [()] (1, 1)
*** exec  0.02 GB    0.11 ms op: MovementOps.EXPAND   out(float): (1024, 1024)                   in(1): [(1, 1)] (1024, 1024)
*** exec  0.03 GB    0.12 ms op: MovementOps.SHRINK   out(float): (1, 1024, 1024)              